# GMPH User Analysis


In [1]:
from datetime import datetime as dt
time = dt.now
import numpy as np
import pandas as pd
import matplotlib as plt
import plotly.offline as py
import plotly.graph_objs as go

import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Show / Hide Code</button>''', raw=True)


py.init_notebook_mode(connected=True)


Show / Hide Code

In [63]:
apps = pd.read_csv('../Data/170419_GMPH_applications.csv',  encoding='latin-1')

In [64]:
accepted = apps[apps['Status'] == 'Condition Pending']
submitted = apps[apps['Application Status'].isin(['Submitted', 'Marked Complete'])]

total = apps['Reference Number'].nunique()
totalaccept = accepted['Reference Number'].nunique()
totalsubmitted = submitted['Reference Number'].nunique()

totalcountries = apps['alpha-3'].nunique()
acceptcountries = accepted['alpha-3'].nunique()

print(str(totalsubmitted) + ' submitted applications from ' + str(totalcountries) + ' countries, ' + str(totalaccept) + ' offered a conditional place.'  )


249 submitted applications from 69 countries, 22 offered a conditional place.


In [58]:
def countries (df, column, divide_by=total):
    
    df1 = df.groupby('alpha-3').count()

    data = [dict(
            type = 'choropleth',
            locations = df1.index.values,
            z = df1[column],
            text = round((df1[column]/divide_by)*100, 2),
            colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
                [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(95, 130, 247)"],[1,"rgb(106, 137, 247)"]],
            autocolorscale = False,
            reversescale = True,
            marker = dict(
                line = dict (
                    color = 'rgb(180,180,180)',
                    width = 0.5
                ) )
          ) ]

    layout = dict(
        width = 1000,
        height =  1000,
        geo = dict(
            showframe = False,
            showcoastlines = False,
            projection = dict(
                type = 'mercator'
            )
        )
    )

    fig = dict( data=data, layout=layout )
    plot = py.iplot(fig)
    
    return plot



# We've recieved over 500 applications from over 60 countries, 249 of these have been submitted

### Hover to see total number of submissions per country (top) and as a percentage of all submissions


In [59]:
countries (submitted, 'Reference Number', totalsubmitted)

# So far 22 offers have been made to applicants in 18 countries 

In [61]:
countries (accepted, 'Reference Number',totalaccept)

In [130]:
submitedu = submitted.groupby('Highest education').nunique().sort_values('Reference Number')
acceptedu = accepted.groupby('Highest education').nunique().sort_values('Reference Number')

# 25% of all submissions list 'other qualifications' as their highest award



In [136]:
trace1 = go.Bar(
    x=round((submitedu['Reference Number'] / totalsubmitted)*100, 0),
    y=submitedu.index,
    name='Submitted',
    orientation = 'h'
)


data = [trace1]
layout = go.Layout(
    barmode='group',
    xaxis=dict(
        domain=[0.3, 1],
        title = '% of total'),
    yaxis=dict(
        tickmode = 'linear',
        showticklabels = True
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

### UK BSc are the most likely to be accepted (100% of UK BSc submissions were accepted)

In [138]:
trace1 = go.Bar(
    x=round((acceptedu['Reference Number'] / submitedu['Reference Number'])*100, 0),
    y=acceptedu.index,
    name='Accepted',
    orientation = 'h'
)

data = [trace1]
layout = go.Layout(
    barmode='group',
    xaxis=dict(
        domain=[0.3, 1]),
        title = '% submissions that were later accepted',
    yaxis=dict(
        tickmode = 'linear',
        showticklabels = True
    ),
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')